# People & Places: Named Entity Recognition

A Jupyter Notebook created for a **Reproducible Research Workshop**

(A Collaboration between Dartmouth Library and Research Computing)

[*Click here to view or register for our current list of workshops*](http://dartgo.org/RRADworkshops), including a workshop next week on [**Stylometry**](https://libcal.dartmouth.edu/event/11237111) (the study of a text's or author's style - to discover authorship of anonymous documents, among other things) and other workshops on Bibliometric Analysis, the customized use of Large Language Models, Text Generation in R, and more....

*Created by*:
+ Jeremy Mikecz, Research Data Services (Dartmouth Library)


In this lesson, we will learn how to extract named entities (names of people, places, groups, institutions, etc.) from text files and then analyze the results.




<div class="alert alert-block alert-success" role="alert">
    <p style="color:green"><b>Brainstorming</b>:</p> 
    <p>For example, one goal may be to create a map showing the countries mentioned, and how frequently they were mentioned, in a corpus of texts.</p>
    <p style="color:green">1. What steps do you anticipate we will have to do in order to successfully accomplish this project? List the steps in the markdown cell below:</p>
</div>

step 1: 

step 2:

step 3:

... 


## I. Using spaCy

**spaCy** is a Python package / library designed to enable fast and efficient Natural Language Processing (for more: [What's spaCy?](https://spacy.io/usage/spacy-101#whats-spacy)). In a previous lesson we used the **Natural Language ToolKit** (NLTK), which does similar things. However, where NLTK is largely designed for instruction and research, spaCy is designed for production, including the fast processing of large amounts of text.

It offers a variety of Natural Language Processing (NLP) features, including:
+ pre-processing tools
    + **tokenization** - dividing text into words (and punctuation marks, numbers, etc.)
    + **sentence boundary detection** - dividing texts into sentences
    + **lemmatization** - identifying the root or base form of words
+ Linguistic Annotations
    + **Part-of-speech tags (POS) and dependencies** - tags part of speech (noun, proper noun, verb, adjective, etc.) and dependency (which words modify which other words? adjectives --> nouns; subject --> verb --> object, etc.)
    + **Named Entity Recognition (NER)** - identifying names of objects, whether people, places, organizations, or other "entities" like book or product titles
    + **word vectorization** - word vectors assign numerical values to words placing each into a multi-dimensional space where similar words are found in close proximity to one another

And much more....

### Working with Foreign Languages

+ You can use [spaCy's existing language models](https://spacy.io/usage/models) for languages from English, Spanish, and Mandarin Chinese to Kyrgyz and Yoruba.
+ you can modify one of these existing language models
+ or you can create a new language model from scratch
    



## II. Installing spaCy

If you were to want to run spaCy on your own computer, [the spaCy instructions](https://spacy.io/usage) recommend installing spaCy in a *virtual environment*. After activating a virtual environment, you would run the following in a terminal:

```
python -m venv .env    #to activate already established virtual environment called ".venv"
source .env/bin/activate     #to activate .venv
pip install -U pip setuptools wheel   
pip install -U spacy             #installs spaCy
python -m spacy download en_core_web_sm   #installs English model from spaCy
```

However, **we are going to install spaCy in JupyterHub**. 

2. To do so, we will need to uncomment (remove the "#") the following cells and run on JupyterHub.

In [ ]:
#!pip install -U spacy

In [ ]:
#!python -m spacy download en_core_web_sm

## III. About Named Entity Recognition with spaCy

Basic named entity recognizers commonly identify the following types of entities:

```
place names
person names
group names
miscellaneous / other entities
```

**spaCy**'s NER identifies a wider-range of entities.

Examine the list of [entity types identified by spaCy](https://towardsdatascience.com/explorations-in-named-entity-recognition-and-was-eleanor-roosevelt-right-671271117218) below:

```
PERSON:      People, including fictional.
NORP:        Nationalities or religious or political groups.
FAC:         Buildings, airports, highways, bridges, etc.
ORG:         Companies, agencies, institutions, etc.
GPE:         Countries, cities, states.
LOC:         Non-GPE locations, mountain ranges, bodies of water.
PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
EVENT:       Named hurricanes, battles, wars, sports events, etc.
WORK_OF_ART: Titles of books, songs, etc.
LAW:         Named documents made into laws.
LANGUAGE:    Any named language.
DATE:        Absolute or relative dates or periods.
TIME:        Times smaller than a day.
PERCENT:     Percentage, including ”%“.
MONEY:       Monetary values, including unit.
QUANTITY:    Measurements, as of weight or distance.
ORDINAL:     “first”, “second”, etc.
CARDINAL:    Numerals that do not fall under another type.
```

<div class="alert alert-block alert-success" role="alert">
    <p style="color:green"><b>Brainstorming</b>:</p> 
    <p style="color:green">3. Brainstorm some ways you could use Named Entity Recognition in your research field for the types of texts and documents researchers in that field typically deal with.</p> 
    <p style="color:green">What projects can you envision?</p> 
    <p style="color:green">What questions could you answer?</p>
</div>

## IV. Import Packages

4. First, let's import all necessary Python packages.

In [ ]:
import spacy
import collections
import pandas as pd
from spacy.lang.en.examples import sentences
from spacy import displacy   #for visualizing word types and relationships

5. Second, since we will be working with English texts, we need to import one of spaCy's English models.

In [ ]:
#https://spacy.io/models/en
nlp = spacy.load("en_core_web_sm")

## V. Linguistic Tagging with spaCy

6. Now we can begin experimenting with spaCy's capabilities. Let's start with some pre-loaded sentences included with the spaCy package:

In [ ]:
print(sentences)

7. Above we loaded an English language model and pipeline called "en_core_web_sm" (a **sm**all **En**glish model trained on the **web**) and saved it as "nlp". Next, we can import text into this pipeline and examine the results. Here, we retrieve **part of speech** tags for a sample sentence.

In [ ]:
doc = nlp(sentences[0])                  #try substituting sentence #0 with another sentence
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

<div class="alert alert-info" role="alert">
    <p style="color:blue"><b>Exercises for Part V</b>:</p> 
    <p style="color:blue">8. Apply spaCy's Part-of-Speech (POS) detection to a sentence or short text of your choice. 
    </p>
</div>

9. Here we retrieve some additional info. Do you recognize what each element returns?

In [ ]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

## VI. Applying Named Entity Recognition (NER) in spaCy

11. Now that we have learned how to retrieve part of speech (POS) information, let's experiment with Named Entity Recognition (NER):

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

12. Let's move beyond spaCy's sample sentences and try extracting named entities (NEs) from other texts. Below, I tried some song lyrics:

In [ ]:
#below: partial lyrics from Boyz II Men's "All Around the World"
lyrics = """
London, Paris, Monte Carlo, Germany and Rome
Different places, different faces still it feels like home
China, Holland, Belgium, Rio, Africa, Japan
That's the way we live and we do the best we can
Here we go on another tour on the road again
Feelin' good it's alright
Just enjoying ourselves
Come and take a flight with
Boyz II Men back around the world
And we're comin' through your town
All we do is for you
'Cause you've always been there
And we appreciate you
Keisha, Kelly, Tonya, Stacy, Mica and LaShaun
Kathy, Trina, Carla, Lisa, Cheri, and Diane
All these girls around the world are fly in every land
And it's hard to choose, but there's one for every man
Here we go on another tour on the road again
Feelin' good it's alright
Just enjoying ourselves
Come and take a flight with
Boyz II Men back around the world
And we're comin' through your town
All we do, we do it for you
'Cause you've always been there
And we appreciate you
Houston, Phoenix, Carolina, Jersey, and the Keys
Denver, Boston, Mississippi, Georgia, Tennessee
Dallas, Cleveland, Cali, Philly, New York, and DC
That's the life we live and it's the only life
"""

doc_lyrics = nlp(lyrics)                  
for ent in doc_lyrics.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


### VIb. Reviewing spaCy's language models

To better understand these results, we need to understand what this model was trained on. At the beginning of this notebook we imported the **en_core_web_sm** model. Let's examine [spaCy's documentation for this model](https://spacy.io/models/en). 

<div class="alert alert-block alert-success" role="alert">
    <p style="color:green"><b>13. Reflection</b>:</p> 
    <ol style = "color:green">
        <li>What texts / sources was this model trained on?</li>
        <li>How does it differ from spaCy's other English models?</li>
        <li>How accurate are these models at NER?</li>
    </ol>
</div>


14. We can experiment with different types of text. Below, we will extract NEs from an excerpt from Jack Kerouac's *On the Road* (1957).

In [ ]:
novel_excerpt = """
part one
1

I first met Dean not long after my wife and I split up. I had just gotten over a serious illness that I won’t bother to talk about, except that it had something to do with the miserably weary split-up and my feeling that everything was dead. With the coming of Dean Moriarty began the part of my life you could call my life on the road. Before that I’d often dreamed of going West to see the country, always vaguely planning and never taking off. Dean is the perfect guy for the road because he actually was born on the road, when his parents were passing through Salt Lake City in 1926, in a jalopy, on their way to Los Angeles. First reports of him came to me through Chad King, who’d shown me a few letters from him written in a New Mexico reform school. I was tremendously interested in the letters because they so naively and sweetly asked Chad to teach him all about Nietzsche and all the wonderful intellectual things that Chad knew. At one point Carlo and I talked about the letters and wondered if we would ever meet the strange Dean Moriarty. This is all far back, when Dean was not the way he is today, when he was a young jailkid shrouded in mystery. Then news came that Dean was out of reform school and was coming to New York for the first time; also there was talk that he had just married a girl called Marylou.
One day I was hanging around the campus and Chad and Tim Gray told me Dean was staying in a cold-water pad in East Harlem, the Spanish Harlem. Dean had arrived the night before, the first time in New York, with his beautiful little sharp chick Marylou; they got off the Greyhound bus at 50th Street and cut around the corner looking for a place to eat and went right in Hector’s, and since then Hector’s cafeteria has always been a big symbol of New York for Dean. They spent money on beautiful big glazed cakes and creampuffs.
"""
doc_kerouac = nlp(novel_excerpt)                  
for ent in doc_kerouac.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


<div class="alert alert-info" role="alert">
    <p style="color:blue"><b>Exercises - Part VI</b>:</p> 
    <p style="color:blue">15. Try applying <b>spaCy's named entity recognition (NER)</b> to a text of your choosing. Copy and paste the above code into the cell below, but insert song lyrics, an excerpt from a novel, or other text.</p>
</div>

## VII. Visualize spaCy tagging using displacy

16. We can visualize NEs, relationships between words (aka. "dependencies"), and other linguistic entities using spaCy's visualization tool [displacy](https://spacy.io/universe/project/displacy).

For more on how to use displacy see: https://spacy.io/usage/visualizers. 

In [ ]:
displacy.render(doc, style = "ent")

In [ ]:
displacy.render(doc, style = "dep")

<div class="alert alert-info" role="alert">
    <p style="color:blue"><b>Exercises - Part VII</b>:</p> 
    <p style="color:blue">17. Use the displacy functions to visualize named entities and dependencies for other texts of your choosing.</p>
</div>

## VIII. Extracting entities from one State of the Union address

In previous lessons, we applied basic text analysis methods to a corpus of 233 State of the Union addresses given by Presidents of the United States from 1791 - 2023.

In one of those lessons, we stored all 233 State of the Union addresses in one .tsv file (tab separated values). This will save us from having to read in all 233 text files individually. 

18. Let's begin by opening this tsv file.

In [ ]:
from pathlib import Path
textdir = Path("~/shared/RR-workshop-data/state-of-the-union-dataset")
textdf = pd.read_csv(Path(textdir, "sotudf.tsv"), encoding = "utf-8", sep = "\t", index_col = 0)
textdf = textdf.sort_values(by = ['year'])
textdf.tail()

19. Let's open one recent SOTU address:

In [ ]:
# In the code below, we are opening the 2009 SOTU 
# presidential address (which would have been Obama's first SOTU address)
sotu = textdf[textdf['year'] == 2009]['fulltext'].item()
sotu[:200]


Let's once again review the list of named entities that spaCy can extract for us:

```
PERSON:      People, including fictional.
NORP:        Nationalities or religious or political groups.
FAC:         Buildings, airports, highways, bridges, etc.
ORG:         Companies, agencies, institutions, etc.
GPE:         Countries, cities, states.
LOC:         Non-GPE locations, mountain ranges, bodies of water.
PRODUCT:     Objects, vehicles, foods, etc. (Not services.)
EVENT:       Named hurricanes, battles, wars, sports events, etc.
WORK_OF_ART: Titles of books, songs, etc.
LAW:         Named documents made into laws.
LANGUAGE:    Any named language.
DATE:        Absolute or relative dates or periods.
TIME:        Times smaller than a day.
PERCENT:     Percentage, including ”%“.
MONEY:       Monetary values, including unit.
QUANTITY:    Measurements, as of weight or distance.
ORDINAL:     “first”, “second”, etc.
CARDINAL:    Numerals that do not fall under another type.
```

<div class="alert alert-block alert-success" role="alert">
    <p style="color:green"><b>Brainstorming</b>:</p> 
    <p style="color:green">19. In the markdown cell below, brainstorm some different research questions you could use spaCy's NER to help you answer (given the list of entities it could help you analyze) about a given SOTU address.</p>
    </div>




20. To begin processing this text with spaCy, we will read it into the spaCy **nlp** object.

In [ ]:
doc_sotu = nlp(sotu)



21. We can retrieve named entities from the nlp object using the **.ents** method.

In [ ]:
for i, ent in enumerate(doc_sotu.ents):
    if i < 10:   # here, we are indicating we only want to return the first 10 entities
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

22. To understand what these labels mean, we can run:

In [ ]:
spacy.explain("NORP")

23. We can even print out these explanations with each entity:

In [ ]:
for i, ent in enumerate(doc_sotu.ents):
    if i < 10:   # here, we are indicating we only want to return the first 10 entities
        print(ent.text, ent.start_char, ent.end_char, ent.label_, spacy.explain(ent.label_))

24. We can save the names and labels of these entities into various formats. Below we are saving info about each entity into a tuple and placing those tuples into a list.

In [ ]:
#ents = [(e.text, e.label_, e.kb_id_) for e in doc_sotu.ents]
ents = [(e.text, e.label_, e.start_char, e.end_char) for e in doc_sotu.ents]
print(ents[:10])

25. We may use the start and end character span for each entity to identify it within its context. For example, the code below prints out the entity and the 50 characters immediately preceding and following it:

In [ ]:
char_span = 50
for i, ent in enumerate(ents):
    if i > 10:
        break
    start_char = ent[2] - char_span
    if start_char < 0:
        start_char = 0
    end_char = ent[3] + char_span
    if end_char > len(doc_sotu.text):
        end_char = len(doc_sotu.text)
    print(ent[0], "=", ent[1])
    print(doc_sotu.text[start_char: end_char])

26. Or we can save only one type of entity. Below, we focus only on person names:

In [ ]:
person_names = []
for ent in ents:
    if ent[1] == "PERSON":
        #person_names.append((ent[0], ent[1]))
        person_names.append(ent[0])
person_names[:10]

#list comprehension to produce the same results in one line of code:
#person_names = [ent[0] for ent in ents if ent[1] == "PERSON"]


27. For place names, spaCy offers at least two different types ("GPE" and "LOC") of place name entities. In the code below, we save both.

In [ ]:
place_names = [(ent[0], ent[1]) for ent in ents if ent[1] in ['GPE', 'LOC']]


In [ ]:
place_names[:20]


28. We can create a frequency list of the place names found within this address using the Counter method from the **collections** package.

In [ ]:
pnfreqs = collections.Counter(place_names)

In [ ]:
pnfreqs

<div class="alert alert-info" role="alert">
    <p style="color:blue"><b>Exercises - Part VIII</b>:</p> 
    <p style="color:blue">29. Compile a frequency list of countries mentioned in another SOTU text. Feel free to copy and paste the relevant code above into the cells below, modifying it as needed.</p>
</div>

### IX. Extracting Named Entities from the entire corpus

We can now scale up and extract NEs from the entire SOTU corpus. A researcher interested in identifying and mapping locations mentioned in these addresses, for example, may want to export all place names. They could extract both "GPEs" (geopolitical entities like countries) and "LOC" (other types of place names). But, for the example below, we will focus on GPEs, which are easier to map.

30. First, we will create a function that extracts all GPEs from a given text.



In [ ]:
def extract_placenames (text):
    doc = nlp(text)
    ents = [(e.text, e.label_, e.kb_id_) for e in doc.ents]
    #place_names = [(ent[0], ent[1]) for ent in ents if ent[1] in ['GPE', 'LOC']]
    gpes = [ent[0] for ent in ents if ent[1] == "GPE"]
    return(gpes)

31. Then, we can apply this function to all our texts found in our sotu dataframe. The code below creates a new column called "gpes" that stores a list of the GPEs found within each text.

*Note: This will take several minutes to run across the SOTU corpus of 233 SOTU addresses. It took 5 minutes to complete on my relatively fast laptop*. The apply function is commented out, but when you have 5 - 20 minutes to spare feel free to run it and check out the results

In [ ]:
#textdf['gpes'] = textdf['fulltext'].apply(extract_placenames)

32. If you have created the new "gpes" column with the code above, you may also uncomment out the code below (remove the `"""`) to create one long list of all gpes found within the SOTU corpus.

In [ ]:

""" 
all_gpes = list([a for b in textdf.gpes.tolist() for a in b])
all_gpes = [gpe.replace("\n", " ") for gpe in all_gpes]
print(len(all_gpes))

with open('textdf_gpes.txt', 'w') as f:
    for gpe in all_gpes:
        f.write(f"{gpe}\n") 
"""


33. We can then create a frequency list of these GPEs, which could be used in subsequent efforts to create a map of all countries in the corpus.

In [ ]:
#sotu_gpes_freqs = collections.Counter(all_gpes)
#sotu_gpes_freqs.most_common(20)

34. You may observe some problems here. "The United States", "the United States", "United States", and "America" are considered separated entities. We would want to aggregate them into one entity. We also have a few state and/or city names mixed in with the country names. More data cleaning is necessary. But, it is a good starting point....

## X. Searching for other types of "entities"

There are other entity recognizers available. Some are customized for particular types of information. For example, biomedical researchers have created their own Named Entity Recognition models for patient records. These models automatically extract a patient's age, sex, their symptoms, lab results, procedures they have undergone, allergies they may have have, etc.

As always, there is always a margin of error in such automated recognition. This rate of error increases as the data being read differs from the data the model was trained on. The researcher must then decide:

+ how much margin of error are they willing to tolerate? (i.e. a researcher tracking general trends across a large population could tolerate more error than a doctor trying to decide on a course of treatment for an individual patient)
+ how much time and effort they can devote to 
    + a. retraining the model to increase accuracy, or
    + b. manually or semi-automatically checking and correcting errors as they arise

